In [1]:
import cifar10
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [2]:
cifar10.data_path = "data/CIFAR-10/"

In [3]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [6]:
x_train=images_train.reshape(images_train.shape[0],-1)
x_test=images_test.reshape(images_test.shape[0],-1)
y_train=cls_train
y_test=cls_test

In [7]:
pca=PCA()
pca.fit_transform (x_train)

array([[-6.40101763e+00,  2.72903876e+00,  1.50171052e+00, ...,
        -7.62287826e-04, -1.16359323e-03, -3.49768400e-04],
       [ 8.29783385e-01, -9.49943219e-01,  6.00375260e+00, ...,
         1.08434589e-04,  5.76715387e-04, -7.71005927e-04],
       [ 7.73019978e+00, -1.15221023e+01, -2.75362051e+00, ...,
        -1.04460109e-03, -2.14105740e-04, -3.93469526e-04],
       ...,
       [ 7.07346587e-01, -1.13425212e+01, -3.93914878e-01, ...,
        -3.80341568e-04,  1.86432348e-03,  1.44785229e-03],
       [ 1.13679090e+01, -3.37597765e+00,  5.56619766e+00, ...,
        -4.67096832e-04,  7.44972963e-04, -1.67065056e-03],
       [ 4.19279645e+00, -1.18418848e+00, -4.44596297e+00, ...,
         9.67968258e-04,  2.40363971e-04, -2.80506224e-05]])

In [8]:
total=sum(pca.explained_variance_)

In [9]:
k=0
curr_sum=0
while (curr_sum/total)<0.99:
    curr_sum+=pca.explained_variance_[k]
    k+=1
k

658

In [10]:
pca=PCA(n_components=k, whiten=True)
x_train_pca=pca.fit_transform (x_train)
x_test_pca=pca.transform (x_test)

In [11]:
from sklearn import svm
svc=svm.SVC(C=10000,gamma='auto')
svc.fit(x_train_pca,y_train)

SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
y_predict_svm=svc.predict(x_test_pca)

In [13]:
from sklearn.metrics import accuracy_score
svm_score=accuracy_score(y_test,y_predict_svm)
svm_score

0.5097

In [14]:
import numpy as np
y_predict=np.array(y_predict_svm)
predictions_decoded = []
for i in range (len(y_predict)):
    predictions_decoded.append(class_names[y_predict[i]])
np.savetxt("cifar.csv",predictions_decoded, fmt = '%s')

In [15]:
from keras.models import Sequential
model=Sequential()
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [16]:
model=tf.keras.models.Sequential([
    Conv2D(32, (3,3), activation='relu',input_shape= (32,32,3)),
    MaxPooling2D (2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D (2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D (2,2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(10,activation='softmax')    
])
model.compile (optimizer= Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])

C:\Users\Rohit\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [17]:
from sklearn.preprocessing import OneHotEncoder

enc=OneHotEncoder()
y_train_encoded = enc.fit_transform(y_train.reshape(-1, 1))
y_test_encoded = enc.transform(y_test.reshape(-1, 1))

C:\Users\Rohit\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [ ]:
model.fit(x_train_pca,y_train_encoded,
                    batch_size=100,
                    epochs=50,
                    validation_data=(x_test_pca,y_test_encoded)
                    )

In [ ]:
predictions = model.predict(x_test_pca)

In [ ]:
predictions_decoded = []

for x in predictions:
    for i in range(x.shape[0]):
        if x[i] == max(x):
            predictions_decoded.append(i)
            break

In [ ]:
labels = []
for i in range(len(predictions_decoded)):
    labels.append(class_names[predictions_decoded[i]])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
accuracy_score(y_test, predictions_decoded)

In [ ]:
import numpy as np
np.savetxt("cifar_predictions.csv", labels, fmt = '%s')

In [ ]:
cls_test

In [ ]:
labels = []
for i in range(len(cls_test)):
    labels.append(class_names[cls_test[i]])